In [1]:
import unittest
from pydantic import BaseModel, Field, field_validator, model_validator, ConfigDict
import plotly.graph_objects as go
import numpy as np
from typing import List, Optional
import uuid

uses a hexagonal coordinate system

ie, going clockwise around origin, pointy side up

| q | r | s |
 |----|--|--|
1 |-1 |0
1 |0 |-1
0 |1 |-1
-1| 1 |0
-1 |0 |1
0 |-1| 1

q is in the up right orientation  
r is rows in the down orientation  
s is in the up left direction  

two coordinates that are adjacant

https://www.redblobgames.com/grids/hexagons/

In [2]:
from typing import Dict
import heapq
from typing import Set, Tuple


class HexCoordinate(BaseModel):
    q: int
    r: int
    s: int
    @field_validator('s', mode='before')
    @classmethod
    def validate_cube_coordinates(cls, v, values):
        if values.data['q'] + values.data['r'] + v != 0:
            raise ValueError('Invalid cube coordinates')
        return v

    def get_adjacent_hexes(self):
        directions = [(1, -1, 0), (1, 0, -1), (0, 1, -1), (-1, 1, 0), (-1, 0, 1), (0, -1, 1)]
        adjacent = []
        for dq, dr, ds in directions:
            adjacent.append(HexCoordinate(q=self.q + dq, r=self.r + dr, s=self.s + ds))
        return adjacent

class GamePiece(BaseModel):
    model_config = ConfigDict(validate_assignment=True)
    
    hex_coordinates: Optional[HexCoordinate] = None
    icon: str = "�"
    team: str
    piece_id: int = Field(default_factory=lambda: str(uuid.uuid4()))
    location: str = Field(default='offboard') # 'offboard', 'board', 'stacked'

    @field_validator('team')
    @classmethod
    def validate_team(cls, v):
        if v not in ['black', 'white']:
            raise ValueError('Team must be either "black" or "white"')
        return v

class Spider(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🕷️",
            location='offboard'
            )

class Ant(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🐜",
            location='offboard'
            )

class Beetle(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🪲",
            location='offboard'
            )

class Grasshopper(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🦗",
            location='offboard'
            )

class QueenBee(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🐝",
            location='offboard'
            )

class Ladybug(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🐞",
            location='offboard'
            )

class Mosquito(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🦟",
            location='offboard'
            )

class BoardState(BaseModel):
    pieces: dict = Field(default_factory=dict)

    def add_piece(self, piece_id: str, piece: GamePiece, coordinates: HexCoordinate):
        piece.hex_coordinates = coordinates
        piece.location = 'board'
        self.pieces[piece_id] = piece

    def move_piece(self, piece_id: str, piece: GamePiece, new_coordinates: HexCoordinate):
        piece.hex_coordinates = new_coordinates
        self.pieces[piece_id] = piece

        # Note: This does not handle stacking logic for Beetles or other pieces.
        # TODO maybe some cool animation here
        
    def get_piece(self, piece_id: str):
        return self.pieces.get(piece_id, None)
    
class Player(BaseModel):
    name: str
    team: str
    pieces: List[GamePiece] = Field(default_factory=None)

    def __init__(self, name: str, team: str, pieces: Optional[List[GamePiece]] = None):
        if pieces is None:
            pieces = [
                *[Ant(team=team) for _ in range(12)],
                QueenBee(team=team)
                # super basic mode to start. add more pieces later
            ]
        super().__init__(name=name, team=team, pieces=pieces)

    @field_validator('team')
    @classmethod
    def validate_team(cls, v):
        if v not in ['black', 'white']:
            raise ValueError('Team must be either "black" or "white"')
        return v

class GameState(BaseModel):
    turn: int = Field(0, ge=0)
    white_player: Player = Player(name='white', team='white', pieces=[])
    black_player: Player = Player(name='black', team='black', pieces=[])
    current_team: str = Field(default='white')
    board_state: BoardState = Field(default_factory=BoardState)
    verbose: bool = Field(default=True)
    all_pieces: Dict[str, GamePiece] = Field(default_factory=dict)

    def __init__(self, white_player: Player = None, black_player: Player = None, **data):
        if white_player is None:
            white_player = Player(name='white', team='white')
        if black_player is None:
            black_player = Player(name='black', team='black')
        super().__init__(white_player=white_player, black_player=black_player, **data)

        # construct a big ol' dictionary of all pieces for easy access
        for piece in self.white_player.pieces:
            self.all_pieces[piece.piece_id] = piece
        for piece in self.black_player.pieces:
            self.all_pieces[piece.piece_id] = piece

    def get_occupied_spaces(self):
        occupied = []
        for piece in self.board_state.pieces.values():
            if piece.location == 'offboard':    
                pass
            elif piece.location == 'board':
                occupied.append((piece.hex_coordinates.q, piece.hex_coordinates.r, piece.hex_coordinates.s))

        return occupied

    def get_available_spaces(self):
        if len(self.board_state.pieces) == 0:
            if self.verbose:
                print("No pieces on the board, returning center hex (0,0,0) as available space.")
            return [HexCoordinate(q=0, r=0, s=0)]  # If no pieces are on the board, return the center hex
        
        # get occupied spaces
        occupied = set(self.get_occupied_spaces())
        adjacent = set()

        for q, r, s in occupied:
            for dq, dr, ds in [(1, -1, 0), (1, 0, -1), (0, 1, -1), (-1, 1, 0), (-1, 0, 1), (0, -1, 1)]:
                adjacent.add((q + dq, r + dr, s + ds))
        relative = adjacent - occupied
        coords = []
        for coord in relative:
            coords.append(HexCoordinate(q=coord[0], r=coord[1], s=coord[2]))
        return coords

    def get_movable_pieces(self, game_state) -> dict:
        """Get all pieces on the board for this bot's team."""
        player = game_state.white_player if self.current_team == 'white' else game_state.black_player
        
        movable = {}
        for piece in player.pieces:
            if piece.location == 'board':  # Only pieces ON the board
                piece_type = piece.__class__.__name__.lower()                
                if piece_type not in movable:
                    movable[piece_type] = []
                movable[piece_type].append(piece.piece_id)
        
        return movable

    def check_win_condition(self):
        # Check if either queen bee is completely surrounded
        white_queen = next((p for p in self.white_player.pieces if isinstance(p, QueenBee)), None)
        black_queen = next((p for p in self.black_player.pieces if isinstance(p, QueenBee)), None)
        if white_queen and white_queen.location == 'board':
            white_adjacent = set((hex.q, hex.r, hex.s) for hex in white_queen.hex_coordinates.get_adjacent_hexes())
            occupied = set(self.get_occupied_spaces())
            if white_adjacent.issubset(occupied):
                return 'black'  # Black wins
        if black_queen and black_queen.location == 'board':
            black_adjacent = set((hex.q, hex.r, hex.s) for hex in black_queen.hex_coordinates.get_adjacent_hexes())
            occupied = set(self.get_occupied_spaces())
            if black_adjacent.issubset(occupied):
                return 'white'  # White wins
        return None  # No winner yet

    def get_queen(self, team: str) -> Optional[QueenBee]:
        """Get the queen bee for a specific team."""
        for piece in self.all_pieces.values():
            if isinstance(piece, QueenBee) and piece.team == team:
                return piece
        return None
    
    def check_queen_placement_loss(self) -> Optional[str]:
        """Check if a player has lost by being unable to place their queen by turn 4."""
        current_player = self.white_player if self.current_team == 'white' else self.black_player
        player_turn_number = self.turn // 2
        
        # Must be at turn 4+ for this player
        if player_turn_number < 4:
            return None
        
        # Check if queen is still offboard
        queen = self.get_queen(self.current_team)
        if not queen or queen.location != 'offboard':
            return None  # Queen already placed, no issue
        
        # Check if there are any valid spaces to place the queen
        available_spaces = self.get_available_spaces()
        for space in available_spaces:
            try:
                Turn.validate_placement(Turn(
                    player=self.current_team,
                    piece_type='queenbee',
                    action_type='place',
                    target_coordinates=space
                ), self)
                return None  # Found at least one valid space
            except ValueError:
                continue  # This space is invalid, keep checking
        
        # No valid spaces found - opponent wins
        opponent = 'black' if self.current_team == 'white' else 'white'
        return opponent

    def get_pieces_by_type(self, piece_type: type, team: Optional[str] = None) -> List[GamePiece]:
        """Get all pieces of a specific type, optionally filtered by team."""
        pieces = [p for p in self.all_pieces.values() if isinstance(p, piece_type)]
        if team:
            pieces = [p for p in pieces if p.team == team]
        return pieces

    def get_piece_by_coordinates(self, coordinates: HexCoordinate) -> Optional[GamePiece]:
        """Get the piece located at specific hex coordinates."""
        for piece in self.all_pieces.values():
            if piece.hex_coordinates == coordinates and piece.location == 'board':
                return piece
        print("No piece found at the given coordinates.")
        return None

    def are_hexes_adjacent(self, hex1: HexCoordinate, hex2: HexCoordinate) -> bool:
        """Check if two hexes are adjacent to each other."""
        # Manhattan distance of 2
        distance = abs(hex1.q - hex2.q) + abs(hex1.r - hex2.r) + abs(hex1.s - hex2.s)
        return distance == 2

    def can_slide_to(self, from_hex: HexCoordinate, to_hex: HexCoordinate, occupied: Set[Tuple[int, int, int]]) -> bool:
        """
        Can slide one tile adjacent, for usage during pathing to check every step is valid.
        """

        # double check path is adjacent
        if not self.are_hexes_adjacent(from_hex, to_hex):
            raise ValueError('Hexes are not adjacent')

        # # coords
        # from_coords = (from_hex.q, from_hex.r, from_hex.s)
        # to_coords = (to_hex.q, to_hex.r, to_hex.s)

        # neighbors
        from_neighbors = set([(h.q, h.r, h.s) for h in from_hex.get_adjacent_hexes()])
        to_neighbors = set([(h.q, h.r, h.s) for h in to_hex.get_adjacent_hexes()])

        mutual_neighbors = from_neighbors.intersection(to_neighbors)
        occupied_mutual = mutual_neighbors.intersection(occupied)

        if len(occupied_mutual) >= 2:
            return False # cannot slide through tight gap
        
        # if len(occupied_mutual) == 0:
        #     # Check if we're sliding along the edge of the hive
        #     all_neighbors = from_neighbors.union(to_neighbors)
        #     all_neighbors.discard(from_coords)  # Remove the from position
        #     all_neighbors.discard(to_coords)    # Remove the to position
            
        #     non_mutual_occupied = all_neighbors.intersection(occupied) - mutual_neighbors
            
        #     # If there are no occupied pieces touching either position (except mutual neighbors),
        #     # the piece would lose contact during the slide
        #     if len(non_mutual_occupied) == 0:
        #         return False
        
        return True

    def get_valid_slide_positions(self, current: HexCoordinate, occupied: Set[Tuple[int, int, int]]) -> List[HexCoordinate]:
        """
        Get all positions that a piece can slide to from the current position.
        A piece can slide to an adjacent empty space if:
        1. The space is not occupied
        2. The piece can physically slide there (not blocked by a gate)
        3. The space is adjacent to at least one other piece (maintains hive connection)
        """
        valid_positions = []
        current_coords = (current.q, current.r, current.s)
        
        for adjacent_hex in current.get_adjacent_hexes():
            adj_coords = (adjacent_hex.q, adjacent_hex.r, adjacent_hex.s)
            
            # 1. Skip if position is occupied
            if adj_coords in occupied:
                continue

            # 2. Check if we can physically slide to this position
            if not self.can_slide_to(current, adjacent_hex, occupied):
                continue

            # 3. Check if this position maintains hive connection
            # (must be adjacent to at least one piece after the move)
            has_neighbor = False
            for neighbor_hex in adjacent_hex.get_adjacent_hexes():
                neighbor_coords = (neighbor_hex.q, neighbor_hex.r, neighbor_hex.s)
                # Don't count the current position as a neighbor (we're leaving it)
                if neighbor_coords in occupied and neighbor_coords != current_coords:
                    has_neighbor = True
                    break
            
            if has_neighbor:
                valid_positions.append(adjacent_hex)
        return valid_positions

    def check_freedom_of_movement(self, start:HexCoordinate, end:HexCoordinate, piece_id: str) -> bool:
        # check if its no move just in case of madness
        if start.q == end.q and start.r == end.r and start.s == end.s:
            return True
        
        # get occupied spaces except the moving piece
        occupied = set()
        for pid, piece in self.board_state.pieces.items():
            if pid != piece_id and piece.location == 'board':
                occupied.add((piece.hex_coordinates.q, piece.hex_coordinates.r, piece.hex_coordinates.s))

        # check if its an ajacent move
        if self.are_hexes_adjacent(start, end):
            return self.can_slide_to(start, end, occupied) # can slide directly
        
        # use a star pathfinding to see if a path exists
        path = self.get_path(start, end, piece_id)
        if path is not None:
            return True
        return False

    def get_path(self, start: HexCoordinate, end: HexCoordinate, piece_id: str) -> Optional[List[HexCoordinate]]:

        def heuristic(a: HexCoordinate, b: HexCoordinate) -> int:
            # Manhattan distance in hex coordinates
            return (abs(a.q - b.q) + abs(a.r - b.r) + abs(a.s - b.s)) // 2
        
        # Priority queue: (f_score, counter, current_hex, path)
        # counter is used to break ties in f_score
        counter = 0
        open_set = []
        heapq.heappush(open_set, (heuristic(start, end), counter, start, [start]))
        
        # Track visited nodes to avoid cycles
        visited = set()
        visited.add((start.q, start.r, start.s))
        
        # Get all occupied spaces except the moving piece
        occupied = set()
        for pid, piece in self.board_state.pieces.items():
            if pid != piece_id and piece.location == 'board':
                occupied.add((piece.hex_coordinates.q, piece.hex_coordinates.r, piece.hex_coordinates.s))
        
        while open_set:
            _, _, current, path = heapq.heappop(open_set)
            
            # Check if we reached the goal
            if current.q == end.q and current.r == end.r and current.s == end.s:
                return path
            
            # Explore neighbors
            for next_hex in self.get_valid_slide_positions(current, occupied):
                next_coords = (next_hex.q, next_hex.r, next_hex.s)
                
                if next_coords not in visited:
                    visited.add(next_coords)
                    g_score = len(path)  # Cost from start to current
                    h_score = heuristic(next_hex, end)  # Heuristic cost to goal
                    f_score = g_score + h_score
                    
                    counter += 1
                    new_path = path + [next_hex]
                    heapq.heappush(open_set, (f_score, counter, next_hex, new_path))
        
        return None 

    @model_validator(mode='after')
    def validate_current_team(self):
        if self.current_team not in ['white', 'black']:
            raise ValueError('Current team must be either "white" or "black"')
        return self

class Turn(BaseModel):
    player: str
    piece_id: Optional[str] = None
    piece_type: Optional[str] = None
    action_type: str # 'place', 'move', 'forfeit'
    target_coordinates: Optional[HexCoordinate] = None

    @staticmethod
    def hive_stays_connected(piece_id, game_state):
        # BFS or DFS to check if all pieces are still connected without the piece being moved
        # get all pieces on board except the one being moved
        pieces_on_board = {}
        for pid, piece in game_state.board_state.pieces.items():
            if pid == piece_id:
                continue
            if piece.location == 'board':
                pieces_on_board[pid] = piece
        
        if len(pieces_on_board) <= 1:
            return True # only one piece on board, so can't break hive
        
        # start BFS from any piece
        # aka, can i go from this one peice to every other piece
        start_id = next(iter(pieces_on_board.keys()))
        visited = {start_id}
        queue = [start_id]

        # build adjacancy map
        coord_to_pid = {}
        for pid, piece in pieces_on_board.items():
            coords = (piece.hex_coordinates.q, piece.hex_coordinates.r, piece.hex_coordinates.s)
            coord_to_pid[coords] = pid

        while queue:
            current_id = queue.pop(0)
            current_piece = pieces_on_board[current_id]

            # check all the adcajacent pieces
            for adj_hex in current_piece.hex_coordinates.get_adjacent_hexes():
                adj_coords = (adj_hex.q, adj_hex.r, adj_hex.s)

                # find all the pieces adjacent to this piece
                if adj_coords in coord_to_pid:
                    adj_pid = coord_to_pid[adj_coords]
                    if adj_pid not in visited:
                        visited.add(adj_pid)
                        queue.append(adj_pid)
        return len(visited) == len(pieces_on_board) # ie if we visited every piece, hive is intact

    @staticmethod
    def validate_movement(turn, game_state):
        # generic movement validation (non-specific to piece type)
        # have to know what id to move
        if turn.piece_id is None:
            raise ValueError('Movement requires piece_id to specify which piece to move')
        
        piece = game_state.all_pieces.get(turn.piece_id)
        if piece is None: # wrong id
            raise ValueError('Piece not found')
        if piece.team != turn.player: # hands off not yours
            raise ValueError('Cannot move opponent piece')
        if piece.location != 'board': # not on board
            raise ValueError('Can only move pieces that are on the board')

        # Check if the target coordinates are valid
        if turn.target_coordinates is None:
            raise ValueError('Movement requires target_coordinates to specify where to move the piece')
        
        # broken hive rule
        if not Turn.hive_stays_connected(turn.piece_id, game_state):
            raise ValueError('Move would break the hive, which is not allowed')

        # freedom of movement rule
        if not game_state.check_freedom_of_movement(piece.hex_coordinates, turn.target_coordinates, turn.piece_id):
            raise ValueError('Piece cannot slide to target coordinates due to freedom of movement rule')

        return turn
    
    @staticmethod
    def validate_placement(turn, game_state):
        
        # need either id or type, find an id if not given
        if turn.piece_id is None: 
            if turn.piece_type is None:
                raise ValueError('Placement requires either piece_id or piece_type to specify which piece to place')
            
            # find an unplaced piece of that type for that player
            piece_type_map = {
                'ant': Ant,
                'beetle': Beetle,
                'grasshopper': Grasshopper,
                'queenbee': QueenBee,
                'queen': QueenBee,
                'spider': Spider,
                'ladybug': Ladybug,
                'mosquito': Mosquito
            }
            piece_class = piece_type_map.get(turn.piece_type.lower())
            
            player = game_state.white_player if turn.player == 'white' else game_state.black_player
            available_piece = next(
                (p for p in player.pieces 
                 if isinstance(p, piece_class) and p.location == 'offboard'),
                None
            )
            # player = game_state.white_player if turn.player == 'white' else game_state.black_player

            if available_piece is None:
                raise ValueError(f'No unplaced piece of type {turn.piece_type} available for player {turn.player}')
            
            turn.piece_id = available_piece.piece_id
        
        # First piece must be placed at the center
        if game_state.turn == 0:
            if turn.target_coordinates != HexCoordinate(q=0, r=0, s=0):
                raise ValueError('First piece must be placed at the center (0,0,0)')
            else:
                return turn
        
        # check its next to an occupied space
        occupied = game_state.get_occupied_spaces()
        adjacent = turn.target_coordinates.get_adjacent_hexes()
        adjacent = [(hex.q, hex.r, hex.s) for hex in adjacent]
        occupied = set(occupied)
        adjacent = set(adjacent)
        if len(occupied.intersection(adjacent)) == 0:
            raise ValueError('Target coordinates must be adjacent to an occupied space')

        target = (turn.target_coordinates.q, turn.target_coordinates.r, turn.target_coordinates.s)
        occupied = set(game_state.get_occupied_spaces())

        if target in occupied:
            raise ValueError('Target coordinates are already occupied')

        # check its not next to an opposite colour
        if game_state.turn > 1: # skip this check for the first placement
            # get players
            player = game_state.white_player if turn.player == 'white' else game_state.black_player
            opponent = game_state.black_player if turn.player == 'white' else game_state.white_player
            
            # get ids
            player_piece_ids = [piece.piece_id for piece in player.pieces if piece.location == 'board']
            opponent_piece_ids = [piece.piece_id for piece in opponent.pieces if piece.location == 'board']

            # coordinates adjacent to player
            player_adjacent = set()
            for pid in player_piece_ids:
                piece = game_state.board_state.pieces[pid]
                for adj in piece.hex_coordinates.get_adjacent_hexes():
                    player_adjacent.add((adj.q, adj.r, adj.s))
            
            # coordinates adjacent to opponent
            opponent_adjacent = set()
            for pid in opponent_piece_ids:
                piece = game_state.board_state.pieces[pid]
                for adj in piece.hex_coordinates.get_adjacent_hexes():
                    opponent_adjacent.add((adj.q, adj.r, adj.s))

            if target not in player_adjacent:
                raise ValueError('Target coordinates must be adjacent to your own pieces')
        
            # Must NOT be adjacent to any opponent pieces
            if target in opponent_adjacent:
                raise ValueError('Target coordinates cannot be adjacent to opponent pieces')

        # check piece is offboard
        # get piece
        piece = game_state.all_pieces.get(turn.piece_id)

        if piece is None:
            raise ValueError('Piece not found in player pieces')
        if piece.location != 'offboard':
            raise ValueError('Piece is already on the board')
        
        # check if queen has been placed by turn 4
        queen = game_state.get_queen(turn.player)
        if turn.player == 'white':
            player_turn_number = game_state.turn // 2
        else:
            player_turn_number = (game_state.turn - 1) // 2

        if player_turn_number == 3 and queen.location == 'offboard':
            # On turn 4, MUST place queen
            piece = game_state.all_pieces.get(turn.piece_id)
            if not isinstance(piece, QueenBee):
                raise ValueError(f'{turn.player.capitalize()}\'s Queen must be placed on turn 4')
        elif player_turn_number > 3 and queen.location == 'offboard':
        # After turn 4, it's too late - this shouldn't happen if enforced properly
            raise ValueError(f'{turn.player.capitalize()} failed to place Queen by turn 4')
    
        return turn

class Game(BaseModel):
    game_state: GameState = Field(default_factory=GameState)
    history: List[Turn] = Field(default_factory=list)

    def apply_turn(self, turn: Turn):
        # Validate turn

        if turn.action_type == 'place':
            turn = Turn.validate_placement(turn, self.game_state)
            
            piece = self.game_state.all_pieces.get(turn.piece_id)
            piece.hex_coordinates = turn.target_coordinates
            piece.location = 'board'
            self.game_state.board_state.add_piece(turn.piece_id, piece, turn.target_coordinates)


        elif turn.action_type == 'move':
            Turn.validate_movement(turn, self.game_state)
            
            # actually move the piece
            piece = self.game_state.all_pieces.get(turn.piece_id)
            piece.hex_coordinates = turn.target_coordinates
            self.game_state.board_state.move_piece(turn.piece_id, piece, turn.target_coordinates)

        elif turn.action_type == 'forfeit':
            if self.game_state.verbose:
                print(f"{turn.player} has forfeited the game.")
            # Forfeit logic to be implemented
            pass
        
        else:
            raise ValueError('Invalid action type')
        
        # Update game state for next turn
        self.history.append(turn)
        win = self.game_state.check_win_condition()
        self.game_state.turn += 1
        self.game_state.current_team = 'black' if self.game_state.current_team == 'white' else 'white'
        return turn.piece_id


In [3]:
from abc import ABC, abstractmethod
import random

class BaseBot(ABC):    
    def __init__(self, team: str, name: str = "Bot"):
        self.team = team
        self.name = name
    
    def get_available_pieces(self, game_state) -> dict:
        """Get all available pieces for this bot's team."""
        player = game_state.white_player if self.team == 'white' else game_state.black_player
        
        available = {}
        for piece in player.pieces:
            if piece.location == 'offboard':
                piece_type = piece.__class__.__name__.lower()
                if piece_type not in available:
                    available[piece_type] = []
                available[piece_type].append(piece.piece_id)
        
        return available
    
    def must_place_queen(self, game_state) -> bool:
        """Check if the bot must place the queen this turn."""
        queen = game_state.get_queen(self.team)
        if self.team == 'white':
            player_turn_number = game_state.turn // 2
        else:
            player_turn_number = (game_state.turn - 1) // 2
        return queen and queen.location == 'offboard' and player_turn_number >= 3


    @abstractmethod
    def choose_action_type(self, can_move: bool, can_place: bool, game_state) -> str:
        """Decide whether to 'move' or 'place'. Must be implemented by subclass."""
        pass
    
    @abstractmethod
    def choose_piece_type(self, available_pieces: dict, movable_pieces: dict, 
                          action_type: str, game_state) -> str:
        """Choose which type of piece to use. Must be implemented by subclass."""
        pass
    
    @abstractmethod
    def choose_piece_id(self, piece_ids: List[str], piece_type: str, 
                        action_type: str, game_state) -> str:
        """Choose specific piece instance from available pieces of chosen type."""
        pass
    
    @abstractmethod
    def choose_target_location(self, available_spaces: List, piece_type: str, 
                               action_type: str, game_state):
        """Choose where to place or move the piece."""
        pass

    def get_move(self, game_state) -> 'Turn':
        """Generate a move..."""
        
        available_pieces = self.get_available_pieces(game_state)
        movable_pieces = game_state.get_movable_pieces(game_state)
        
        if not available_pieces and not movable_pieces:
            return Turn(player=self.team, action_type='forfeit')
        
        # Check if must place the queen
        if self.must_place_queen(game_state):
            action_type = 'place'
            piece_type = 'queenbee'
        else:
            # Delegate decision
            can_move = len(movable_pieces) > 0
            can_place = len(available_pieces) > 0
            
            if not can_move and not can_place:
                return Turn(player=self.team, action_type='forfeit')
            
            action_type = self.choose_action_type(can_move, can_place, game_state)
            piece_type = self.choose_piece_type(available_pieces, movable_pieces, 
                                                action_type, game_state)
        
        available_spaces = game_state.get_available_spaces()
        
        if action_type == 'move':
            # Filter for pieces that won't break the hive
            candidate_pieces = movable_pieces.get(piece_type, [])
            valid_moves = {}

            for pid in candidate_pieces:
                if not Turn.hive_stays_connected(pid, game_state):
                    continue
                    
                piece = game_state.all_pieces.get(pid)
                valid_targets = []
                for space in available_spaces:
                    try:
                        test_turn = Turn(
                            player=self.team,
                            piece_id=pid,
                            action_type='move',
                            target_coordinates=space
                        )
                        Turn.validate_movement(test_turn, game_state)
                        valid_targets.append(space)
                    except ValueError:
                        continue
                if valid_targets:
                    valid_moves[pid] = valid_targets

            if not valid_moves:
                # Fall back to placement
                action_type = 'place'
                if available_pieces:
                    piece_type = self.choose_piece_type(available_pieces, {}, 
                                                    'place', game_state)
                else:
                    return Turn(player=self.team, action_type='forfeit')
            else:
                pieces_with_moves = list(valid_moves.keys())
                piece_id = self.choose_piece_id(pieces_with_moves, 
                                            piece_type, action_type, game_state)
                
                target = self.choose_target_location(valid_moves[piece_id], piece_type, 
                                                    action_type, game_state)

                return Turn(
                    player=self.team,
                    piece_id=piece_id,
                    action_type='move',
                    target_coordinates=target
                )
        
        # Handle placement (or fallback from failed move)
        if action_type == 'place':  
            valid_spaces = []
            for space in available_spaces:
                try:
                    Turn.validate_placement(Turn(
                        player=self.team,
                        piece_type=piece_type,
                        action_type='place',
                        target_coordinates=space
                    ), game_state)
                    valid_spaces.append(space)
                except ValueError:
                    pass  # Invalid space, skip it
            
            if not valid_spaces:
                if game_state.verbose:
                    print(f"No valid placement spaces for {piece_type}")
                    print(f"Available spaces checked: {len(available_spaces)}")
                return Turn(player=self.team, action_type='forfeit')
            
            # Choose specific piece and target
            piece_id = self.choose_piece_id(available_pieces[piece_type], 
                                        piece_type, 'place', game_state)
            target = self.choose_target_location(valid_spaces, piece_type, 
                                                'place', game_state)
            
            return Turn(
                player=self.team,
                action_type='place',
                piece_type=piece_type,
                piece_id=piece_id,
                target_coordinates=target
            )


class RandomBot(BaseBot):
    """A bot that makes completely random valid moves."""
    
    def __init__(self, team: str, name: str = "RandomBot"):
        super().__init__(team, name)
    
    def choose_action_type(self, can_move: bool, can_place: bool, game_state) -> str:
        """Randomly choose between move and place."""
        options = []
        if can_move:
            options.append('move')
        if can_place:
            options.append('place')
        return random.choice(options)
    
    def choose_piece_type(self, available_pieces: dict, movable_pieces: dict,
                          action_type: str, game_state) -> str:
        """Randomly select a piece type based on action."""
        if action_type == 'move':
            return random.choice(list(movable_pieces.keys()))
        else:
            return random.choice(list(available_pieces.keys()))
    
    def choose_piece_id(self, piece_ids: List[str], piece_type: str,
                        action_type: str, game_state) -> str:
        """Randomly select a specific piece."""
        return random.choice(piece_ids)
    
    def choose_target_location(self, available_spaces: List, piece_type: str,
                               action_type: str, game_state):
        """Randomly select an available space."""
        return random.choice(available_spaces)

In [4]:
def hex_to_pixel(coord: HexCoordinate, size: float = 1.0):
    """Convert hex coordinate to pixel position for plotting."""
    x = size * (3/2 * coord.q)
    y = size * (np.sqrt(3)/2 * coord.q + np.sqrt(3) * coord.r)
    return x, y

def get_hexagon_vertices(x: float, y: float, size: float = 1.0):
    """Get vertices of a hexagon centered at (x, y)."""
    angles = np.linspace(0, 2*np.pi, 7)  # 7 points to close the hexagon
    vertices_x = x + size * np.cos(angles)
    vertices_y = y + size * np.sin(angles)
    return vertices_x, vertices_y

def visualize_game_board(board_state: BoardState, show_empty_hexes: Optional[List[HexCoordinate]] = None, show_coordinates: bool = True):
    """
    Visualize game pieces on hex coordinates
    """
    fig = go.Figure()
    
    hex_size = 0.95
    icon_size = int(25 * hex_size)
    
    # CUSTOMIZATION: Team colors mapping
    team_colors = {
        "black": "#1D1A1A",  # black team
        "white": "#FFFFFF",  # white team
    }
    
    team_border_colors = {
        "black": "#000000",  # black border
        "white": "#808080",  # white border for visibility
    }
    
    # Draw empty hexes if provided
    if show_empty_hexes:
        for coord in show_empty_hexes:
            center_x, center_y = hex_to_pixel(coord, size=1.0)
            hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
            
            fig.add_trace(go.Scatter(
                x=hex_x,
                y=hex_y,
                fill='toself',
                fillcolor='#F5F5F5',
                line=dict(color='lightgray', width=1),
                mode='lines',
                showlegend=False,
                name='',
                hovertemplate=f'Empty<br>q={coord.q}, r={coord.r}, s={coord.s}<extra></extra>',
            ))
            
            if show_coordinates:
                fig.add_trace(go.Scatter(
                    x=[center_x],
                    y=[center_y],
                    mode='text',
                    text=[f'({coord.q},{coord.r},{coord.s})'],
                    textfont=dict(size=10, color='darkgray'),
                    showlegend=False,
                    name='',
                    hoverinfo='skip'
                ))
        
    # Draw hexes with game pieces (only pieces that are on the board)
    for piece in board_state.pieces.values():
        # Skip pieces without coordinates (offboard pieces)
        if piece.hex_coordinates is None:
            continue
            
        # Skip pieces that are explicitly offboard
        if piece.location == 'offboard':
            continue
        
        coord = piece.hex_coordinates
        center_x, center_y = hex_to_pixel(coord, size=1.0)
        hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
        
        # Get team colors
        fill_color = team_colors.get(piece.team, 'lightgray')
        line_color = team_border_colors.get(piece.team, 'gray')
        
        # Draw hexagon
        fig.add_trace(go.Scatter(
            x=hex_x,
            y=hex_y,
            fill='toself',
            fillcolor=fill_color,
            line=dict(color=line_color, width=2),
            mode='lines',
            showlegend=False,
            name='',
            hovertemplate=f'{piece.__class__.__name__} ({piece.team})<br>Position: ({coord.q},{coord.r},{coord.s})<extra></extra>',
        ))
        
        # Add piece icon
        fig.add_trace(go.Scatter(
            x=[center_x],
            y=[center_y],
            mode='text',
            text=[piece.icon],
            textfont=dict(size=icon_size, color='black'),
            showlegend=False,
            name='',
            hoverinfo='skip'
        ))
        
        # Add coordinate labels (optional)
        if show_coordinates:
            fig.add_trace(go.Scatter(
                x=[center_x],
                y=[center_y - 0.3],
                mode='text',
                text=[f'({coord.q},{coord.r},{coord.s})'],
                textfont=dict(size=8, color='gray'),
                showlegend=False,
                name='',
                hoverinfo='skip'
            ))

    fig.update_layout(
        title='Hive - digitally made by Dan',
        hovermode='closest',
        xaxis=dict(
            scaleanchor='y',
            scaleratio=1,
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        plot_bgcolor='white',
        width=800,
        height=800,
    )
    
    fig.show()

In [5]:
import time


def simulate_game(white_bot, black_bot, verbose=False, plot_game: bool = False):
    """Simulate a game between two bots."""
    
    game = Game(game_state=GameState(verbose=verbose))
    
    max_turns = 200  # Prevent infinite games
    
    for turn_num in range(max_turns):
        queen_loss = game.game_state.check_queen_placement_loss()
        if queen_loss:
            if verbose:
                print(f"\n{game.game_state.current_team.upper()} cannot place queen by turn 4!")
                print(f"{queen_loss.upper()} WINS by queen placement rule!")
            return queen_loss, turn_num, game

        current_bot = white_bot if game.game_state.current_team == 'white' else black_bot
        
        # Get bot's move
        turn = current_bot.get_move(game.game_state)
        
        if verbose:
            print(f"\nTurn {turn_num}: {current_bot.name} ({current_bot.team})")
            print(f"  Action: {turn.action_type}")
            if turn.piece_type:
                print(f"  Piece: {turn.piece_type}")
            if turn.piece_id and turn.action_type == 'move':
                # Show current location for moves
                piece = game.game_state.all_pieces.get(turn.piece_id)
                if piece and piece.hex_coordinates:
                    print(f"  From: ({piece.hex_coordinates.q}, {piece.hex_coordinates.r}, {piece.hex_coordinates.s})")
            if turn.target_coordinates:
                print(f"  Target: ({turn.target_coordinates.q}, {turn.target_coordinates.r}, {turn.target_coordinates.s})")
        
        # Apply the turn
        try:
            game.apply_turn(turn)
        except Exception as e:
            print(f"Error applying turn: {e}")
            break
        
        # Check for winner
        winner = game.game_state.check_win_condition()
        if winner:
            if verbose:
                print(f"\n{winner.upper()} WINS after {turn_num + 1} turns!")
            return winner, turn_num + 1, game
    
    if verbose:
        print(f"\nGame reached maximum turns ({max_turns})")
    if plot_game:
        visualize_game_board(game.game_state.board_state, show_empty_hexes=game.game_state.get_available_spaces())
        time.sleep(1)
    return None, max_turns, game

In [6]:
# Single game with verbose output
import random


white = RandomBot(team='white')
black = RandomBot(team='black')
_, max_turns, game =simulate_game(white, black, verbose=True, plot_game=True)

visualize_game_board(game.game_state.board_state, show_empty_hexes=game.game_state.get_available_spaces())

No pieces on the board, returning center hex (0,0,0) as available space.

Turn 0: RandomBot (white)
  Action: place
  Piece: ant
  Target: (0, 0, 0)

Turn 1: RandomBot (black)
  Action: place
  Piece: ant
  Target: (0, 1, -1)

Turn 2: RandomBot (white)
  Action: move
  From: (0, 0, 0)
  Target: (-1, 2, -1)

Turn 3: RandomBot (black)
  Action: move
  From: (0, 1, -1)
  Target: (-1, 3, -2)

Turn 4: RandomBot (white)
  Action: place
  Piece: queenbee
  Target: (-2, 2, 0)

Turn 5: RandomBot (black)
  Action: place
  Piece: ant
  Target: (-1, 4, -3)

Turn 6: RandomBot (white)
  Action: place
  Piece: ant
  Target: (-1, 1, 0)

Turn 7: RandomBot (black)
  Action: place
  Piece: queenbee
  Target: (0, 4, -4)

Turn 8: RandomBot (white)
  Action: place
  Piece: ant
  Target: (0, 1, -1)

Turn 9: RandomBot (black)
  Action: place
  Piece: ant
  Target: (1, 3, -4)

Turn 10: RandomBot (white)
  Action: move
  From: (0, 1, -1)
  Target: (0, 2, -2)

Turn 11: RandomBot (black)
  Action: move
  From: (1

In [7]:
def find_valid_placements(game_state, team: str, piece_type: str = None) -> list:
    """Helper to find all valid placement locations for a team"""
    valid_spaces = []
    available_spaces = game_state.get_available_spaces()
    
    for space in available_spaces:
        try:
            # Create a test turn
            test_turn = Turn(
                player=team,
                piece_type=piece_type or 'ant',
                action_type='place',
                target_coordinates=space
            )
            Turn.validate_placement(test_turn, game_state)
            valid_spaces.append(space)
        except ValueError:
            pass
    
    return valid_spaces

def debug_placement_issues(game_state):
    """Comprehensive debug for placement issues"""
    print("\n=== PLACEMENT DEBUG ===")
    print(f"Current turn: {game_state.turn}")
    print(f"Current team: {game_state.current_team}")
    
    # Check queen placement requirements
    for team in ['white', 'black']:
        queen = game_state.get_queen(team)
        if team == 'white':
            turn_num = game_state.turn // 2
        else:
            turn_num = (game_state.turn - 1) // 2
        
        print(f"\n{team.capitalize()} team:")
        print(f"  Player turn number: {turn_num}")
        print(f"  Queen status: {queen.location if queen else 'No queen'}")
        print(f"  Must place queen: {turn_num == 3 and queen.location == 'offboard'}")
        
        # Find valid placements
        valid_placements = find_valid_placements(game_state, team)
        print(f"  Valid placement spots: {len(valid_placements)}")
        
        if len(valid_placements) < 5:  # Show details if few options
            for i, space in enumerate(valid_placements):
                print(f"    {i+1}. ({space.q}, {space.r}, {space.s})")
        
        # Special check for queen placement
        if queen and queen.location == 'offboard':
            queen_valid = find_valid_placements(game_state, team, 'queenbee')
            print(f"  Valid queen placement spots: {len(queen_valid)}")

spaces = debug_placement_issues(game.game_state)
spaces


=== PLACEMENT DEBUG ===
Current turn: 200
Current team: white

White team:
  Player turn number: 100
  Queen status: board
  Must place queen: False
  Valid placement spots: 0

Black team:
  Player turn number: 99
  Queen status: board
  Must place queen: False
  Valid placement spots: 0


In [8]:
game = Game()

id = game.apply_turn(Turn(
    player='white',
    piece_type = 'ant',
    action_type='place', 
    target_coordinates=HexCoordinate(q=0, r=0, s=0)
))

id = game.apply_turn(Turn(
    player='black',
    piece_type = 'ant',
    action_type='place',
    target_coordinates=HexCoordinate(q=1, r=-1, s=0)
))

game.apply_turn(Turn(
    player='white',
    piece_type='ant',
    action_type='place',
    target_coordinates=HexCoordinate(q=-1, r=1, s=0)
))

game.apply_turn(Turn(
    player='black',
    piece_type='ant',
    action_type='place',
    target_coordinates=HexCoordinate(q=2, r=-1, s=-1)
))

whiteid = game.apply_turn(Turn(
    player='white',
    piece_type='queen',
    action_type='place',
    target_coordinates=HexCoordinate(q=0, r=1, s=-1)
))

id = game.apply_turn(Turn(
    player='black',
    piece_type='queen',
    action_type='place',
    target_coordinates=HexCoordinate(q=3, r=-1, s=-2)
))

movepiece = game.game_state.get_piece_by_coordinates(HexCoordinate(q=-1, r=1, s=0))

game.apply_turn(Turn(
    player='white',
    piece_id=movepiece.piece_id,
    action_type='move',
    target_coordinates=HexCoordinate(q=2, r=0, s=-2)
))


visualize_game_board(game.game_state.board_state, show_empty_hexes=game.game_state.get_available_spaces())

# # Check white queen location
# white_queen = game.game_state.get_queen('white')
# if white_queen and white_queen.location == 'board':
#     print(f"White queen is at: ({white_queen.hex_coordinates.q}, {white_queen.hex_coordinates.r}, {white_queen.hex_coordinates.s})")

Turn 0: RandomBot (white)
  Action: place
  Piece: queenbee
  Target: (0, 0, 0)

Turn 1: RandomBot (black)
  Action: place
  Piece: ant
  Target: (1, 0, -1)

Turn 2: RandomBot (white)
  Action: place
  Piece: ant
  Target: (-1, 1, 0)

Turn 3: RandomBot (black)
  Action: move
  Target: (-2, 2, 0)

Turn 4: RandomBot (white)
  Action: place
  Piece: ant
  Target: (0, -1, 1)
No valid placement spaces for ant
Available spaces checked: 12

Turn 5: RandomBot (black)
  Action: forfeit
black has forfeited the game.

Turn 6: RandomBot (white)
  Action: place
  Piece: ant
  Target: (0, -2, 2)
No valid placement spaces for queenbee
Available spaces checked: 14

Turn 7: RandomBot (black)
  Action: forfeit
black has forfeited the game.


In [9]:
game = Game()

id = game.apply_turn(Turn(
    player='white',
    piece_type = 'queen',
    action_type='place', 
    target_coordinates=HexCoordinate(q=0, r=0, s=0)
))

id = game.apply_turn(Turn(
    player='black',
    piece_type = 'ant',
    action_type='place',
    target_coordinates=HexCoordinate(q=1, r=0, s=-1)
))

game.apply_turn(Turn(
    player='white',
    piece_type='ant',
    action_type='place',
    target_coordinates=HexCoordinate(q=-1, r=1, s=0)
))

movepiece = game.game_state.get_piece_by_coordinates(HexCoordinate(q=1, r=0, s=-1))

game.apply_turn(Turn(
    player='black',
    piece_id=movepiece.piece_id,
    action_type='move',
    target_coordinates=HexCoordinate(q=-2, r=2, s=0)
))

whiteid = game.apply_turn(Turn(
    player='white',
    piece_type='ant',
    action_type='place',
    target_coordinates=HexCoordinate(q=0, r=-1, s=1)
))

# id = game.apply_turn(Turn(
#     player='black',
#     piece_type='queen',
#     action_type='place',
#     target_coordinates=HexCoordinate(q=3, r=-1, s=-2)
# ))

# movepiece = game.game_state.get_piece_by_coordinates(HexCoordinate(q=-1, r=1, s=0))

# game.apply_turn(Turn(
#     player='white',
#     piece_id=movepiece.piece_id,
#     action_type='move',
#     target_coordinates=HexCoordinate(q=2, r=0, s=-2)
# ))


visualize_game_board(game.game_state.board_state, show_empty_hexes=game.game_state.get_available_spaces())
